In [ ]:
import librosa

from IPython.display import display, Audio
import glob
from tqdm import tqdm
import random
import math

!pip install pyroomacoustics
import pyroomacoustics as pa
import pandas as pd

from scipy.signal import stft,istft,convolve
import scipy
import numpy as np

     |████████████████████████████████| 1.1MB 3.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached https://files.pythonhosted.org/packages/92/a5/796ccb611a92cbdf1f6eaa9b2ec9458d5784a7b909efdf8e8afaa3861783/pybind11-2.6.0-py2.py3-none-any.whl
  Created wheel for pyroomacoustics: filename=pyroomacoustics-0.4.2-cp36-cp36m-linux_x86_64.whl size=9532844 sha256=281275247a2f3d13220812e5cdf3bd2858ea8d700d3f0244d3e02d286f428119
  Stored in directory: /root/.cache/pip/wheels/a8/36/c1/e2c6c0c9a018975e20f7c4694ee1d19f768ac7b3aba168231a
Successfully built pyroomacoustics


# Datasets
音声　CMU Arctic Corpus

```
# CMU Arctic Corpusのダウンロード(初回のみ)
pa.datasets.CMUArcticCorpus(basedir="./CMU_ARCTIC" ,download=True)
```
雑音 Ernie, UrbanSound8K
```
# n_files1 = glob.glob(noise1+"/disk107/Datasets/noise/ernieNoise/trim_wavs/*wav")#ernie
# n_files2 = glob.glob(noise2+"/disk107/Datasets/UrbanSound8K/audio/fold*/*.wav")#urban
```

# データセットの作成

In [ ]:
def addnoise(c_data,n_data,SNR = 0.1):
    """
    c_data:音声データ
    n_data:ノイズデータ
    
    クリーンな音声データに任意の雑音をつけます。
    
    音声データの長さがノイズデータの長さより長い時→ノイズデータをリピートして音声データの長さに合わせる
    音声データの長さがノイズデータの長さより短い時→ノイズデータを音声な音声データの長さに合わせて切り捨て
    """

    c_data_s = c_data * SNR #ernieノイズの音量が小さいので調整してます。

    if len(c_data) == len(n_data):
        noise_data = c_data_s + n_data

    elif len(c_data) > len(n_data):

        q, mod = divmod(len(c_data), len(n_data))

        if q == 1:
            new = np.append(n_data,n_data[:mod])

        else:
            new = np.append(n_data,n_data)
            for i in range(q-2):
                new = np.append(new,n_data)
            new = np.append(new,n_data[:mod])

        noise_data = c_data_s + new

    else:
        noise_data = c_data_s + n_data[:len(c_data)]
        
    return noise_data

In [ ]:
# clean：CMU Arctic Copus内のwavファイル
c_files = glob.glob("CMU_ARCTIC/*/wav/*.wav")
random.shuffle(c_files)

# noise
n_files1 = glob.glob(noise1+"/disk107/Datasets/noise/ernieNoise/trim_wavs/*wav")#ernie
n_files2 = glob.glob(noise2+"/disk107/Datasets/UrbanSound8K/audio/fold*/*.wav")#urban
n_files = np.append(n_files1,n_files2)#合体
random.shuffle(n_files)

clean = []
noise = []

for n in tqdm(len(c_files)):
    # noise
    c_data, fs = librosa.load(c_files[n],sr=16000)
    n_data, fs = librosa.load(n_files[n],sr=16000)
    
    noise_data = addnoise(c_data,n_data,SNR = 0.7)
    
    clean.append(c_data)
    noise.append(noise_data)

In [ ]:
np.save('dataset', clean)
np.save('dataset', noise)